# Prep some data

In [1]:
import scipy.io as sio
import numpy as np

import matplotlib.pyplot as plt

import time

np.random.seed(69)

In [2]:
datasets = sio.loadmat('../data.mat')

# Remove keys that aren't modulation schemes
del(datasets['__header__'], datasets['__globals__'], datasets['__version__'])

train_data = np.vstack([mod_scheme[0] for dataset in datasets for mod_scheme in datasets[dataset][0]])

print(train_data.shape)

train_data = np.reshape(train_data, [-1, 1, 128, 2])

print(train_data.shape)

print([dataset for dataset in datasets for mod_scheme in datasets[dataset][0]])

# Number of classes
num_classes = len(datasets)
size = int(train_data.shape[0]/num_classes)

train_labels = np.zeros((train_data.shape[0],1))

for i in range(num_classes):
    train_labels[(i)*size:(i+1)*size] = np.ones((size, 1)) * i

# One hot encoding
labels = np.reshape(train_labels, [-1]) # Reshape into (N,) from (N,1), otherwise assignments get messed up
#labels_encoded = np.zeros((labels.shape[0], num_classes))
#labels_encoded[np.arange(labels.shape[0]), labels.astype(int)] = 1

train_labels = labels

print(train_labels.shape)

(2048, 128, 2)
(2048, 1, 128, 2)
['bpsk', 'qpsk']
(2048,)


In [3]:
# Shuffle
indexes = np.arange(train_labels.shape[0])
np.random.shuffle(indexes)

train_data = train_data[indexes]
train_labels = train_labels[indexes]

In [4]:
# Save thing in HDF5
import h5py, os

f = h5py.File('training_data.h5', 'w')
f['data'] = train_data
f['label'] = train_labels
f.close()

text_train = os.path.join('./', 'train_data.txt')
with open(text_train, 'w') as g:
    print('training_data.h5', file = g)
    g.close()

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Do caffe stuff

In [5]:
import caffe

In [6]:
caffe.set_device(0)
caffe.set_mode_cpu()

In [7]:
net = caffe.Net('amc_network.prototxt', caffe.TEST)

In [8]:
[(k, v.data.shape) for k, v in net.blobs.items()]

[('data', (64, 1, 128, 2)),
 ('label', (64,)),
 ('label_data_1_split_0', (64,)),
 ('label_data_1_split_1', (64,)),
 ('conv', (64, 16, 127, 1)),
 ('output', (64, 2032)),
 ('ip1', (64, 1)),
 ('ip1_ip1_0_split_0', (64, 1)),
 ('ip1_ip1_0_split_1', (64, 1)),
 ('loss', ()),
 ('accuracy', ())]

In [9]:
solver = caffe.get_solver('solver.prototxt')

In [10]:
solver.solve()

In [11]:
accuracy = 0
batch_size = solver.test_nets[0].blobs['data'].num
for i in range(4500):
    solver.test_nets[0].forward()
    accuracy += solver.test_nets[0].blobs['accuracy'].data
accuracy /= 4500

In [12]:
accuracy

1.0